<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-shapefile" data-toc-modified-id="Import-shapefile-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import shapefile</a></span></li><li><span><a href="#Extracting-array-from-the-raster" data-toc-modified-id="Extracting-array-from-the-raster-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Extracting array from the raster</a></span><ul class="toc-item"><li><span><a href="#FOR-LOOP" data-toc-modified-id="FOR-LOOP-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>FOR LOOP</a></span></li><li><span><a href="#LAMBDA" data-toc-modified-id="LAMBDA-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LAMBDA</a></span></li><li><span><a href="#List-comprehansion" data-toc-modified-id="List-comprehansion-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>List comprehansion</a></span></li></ul></li><li><span><a href="#What-if-we-need-to-extract-from-multiple-images?" data-toc-modified-id="What-if-we-need-to-extract-from-multiple-images?-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>What if we need to extract from multiple images?</a></span><ul class="toc-item"><li><span><a href="#Loop-over-all-images-with-rasterio" data-toc-modified-id="Loop-over-all-images-with-rasterio-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Loop over all images with rasterio</a></span></li><li><span><a href="#Pre-stack-all-images-into-a-single-(multiband)-file" data-toc-modified-id="Pre-stack-all-images-into-a-single-(multiband)-file-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Pre-stack all images into a single (multiband) file</a></span></li></ul></li></ul></div>

In [1]:
# Import requiered python packages
import geopandas as gpd
from shapely.geometry import mapping

import numpy as np

import rasterio
from rasterio.mask import mask

## Import shapefile

In [2]:
%%time

src = "c:\\Users\\ncoz\\ARRS_susa\\mala_sola\\ZV2017_d96tm_big10.shp"

shape = gpd.read_file(src)

Wall time: 49.8 s


In [3]:
shape.shape

(681968, 10)

In [4]:
shape.head()

,GERK_PID,POLJINA_ID,RABA_ID,POVR_GERK_,SIFRA_KMRS,OBRAZEC,INTERSECT,POVR_ar,REGIJA,geometry
0,2537988,2104906.0,1100,19308.9997,801,GP,0,57.930456,5,"POLYGON ((547508.347 86829.736, 547545.426 868..."
1,2537988,2104908.0,1100,19308.9997,005,GP,0,30.520097,5,"POLYGON ((547497.850 86785.076, 547578.847 867..."
2,2537988,2104910.0,1100,19308.9997,203,GP,0,26.799442,5,"POLYGON ((547599.139 86850.730, 547670.666 868..."
3,1143420,2105003.0,1100,8277.3796,801,GP,0,40.952670,5,"POLYGON ((547165.958 85941.243, 547179.112 859..."
4,3077991,2652377.0,1211,14987.7976,100,GP,0,149.877976,4,"POLYGON ((419052.089 80240.803, 419056.851 802..."


## Extracting array from the raster

Rasterio mask requires the input geometry to be in a specific format (GeoJSON).

This step will reformat the shapely-type geometry into GeoJSON-type and append it to a new column.


In [3]:
%%time

shape["geom"] = shape.geometry.apply(lambda g: [mapping(g)])

Wall time: 37.6 s


In [4]:
shape[["geometry", "geom"]].head()

,geometry,geom
0,"POLYGON ((547508.347 86829.736, 547545.426 868...","[{'type': 'Polygon', 'coordinates': (((547508...."
1,"POLYGON ((547497.850 86785.076, 547578.847 867...","[{'type': 'Polygon', 'coordinates': (((547497...."
2,"POLYGON ((547599.139 86850.730, 547670.666 868...","[{'type': 'Polygon', 'coordinates': (((547599...."
3,"POLYGON ((547165.958 85941.243, 547179.112 859...","[{'type': 'Polygon', 'coordinates': (((547165...."
4,"POLYGON ((419052.089 80240.803, 419056.851 802...","[{'type': 'Polygon', 'coordinates': (((419052...."


Define the function that needs to be applied to each row of the DataFrame

In [5]:
from rasterio.mask import mask

def msk_rio(geom, raster):
    """Returns subset of an array covered by the input polygon. The input
    polygon has to be in the GeoJSON format. The crop attribute sets values
    not covered by polygon to nan. All_touched is used to prevent empty rasters
    for very thin polygons."""
    
    out_image, _ = mask(raster, geom, crop=True, all_touched=True, nodata=np.nan)
    
    return out_image

### FOR LOOP

Prepare a samller subset of polygons for testing. Add geometry in the appropriate format

In [12]:
# shp = shape[:5000].copy()
shp = shape.copy()

In [13]:
%%time

# Open rasterio dataset
ds = rasterio.open("c:\\Users\\ncoz\\ARRS_susa\\fill_tif_fill\\dem_slo_03_HAND_fill_nan.tif")

# Create rcolumn for saving results
shp["hand_mean"] = np.nan

for index, item in shp.iterrows():
    # Extract raster and calculate mean
    sub_raster = msk_rio(item["geom"], ds)
    shp.loc[index, "hand_mean"] = np.nanmean(sub_raster)

shp = shp.drop(columns=["geom"])
ds.close()

C:\ProgramData\Anaconda3\envs\hand_index\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


KeyboardInterrupt: 

In [14]:
shp.tail(3)

,GERK_PID,POLJINA_ID,RABA_ID,POVR_GERK_,SIFRA_KMRS,OBRAZEC,INTERSECT,POVR_ar,REGIJA,geometry,hand_mean
4997,1425714,2119385.0,1100,5183.5005,006,GP,0,51.835005,5,"POLYGON ((566513.783 156266.730, 566511.996 15...",9.497029
4998,3319963,2119386.0,1300,5424.2931,204,GP,0,54.242931,5,"POLYGON ((566610.648 156430.799, 566627.841 15...",8.743620
4999,2352253,2119388.0,1100,7313.3327,206,GP,0,73.133327,5,"POLYGON ((566268.004 156617.728, 566294.294 15...",12.367579


### LAMBDA

Reset table

In [9]:
shp = shape.copy()

In [10]:
%%time

# Open rasterio dataset
ds = rasterio.open("c:\\Users\\ncoz\\ARRS_susa\\fill_tif_fill\\dem_slo_03_HAND_fill_nan.tif")

shp["sub_raster"] = shp["geom"].apply(lambda g: msk_rio(g, ds))
shp["hand_mean"] = shp["sub_raster"].apply(lambda g: np.nanmean(g))

ds.close()
shp = shp.drop(columns=["geom", "sub_raster"])

Wall time: 19min 29s


In [11]:
shp.tail(3)

,GERK_PID,POLJINA_ID,RABA_ID,POVR_GERK_,SIFRA_KMRS,OBRAZEC,INTERSECT,POVR_ar,REGIJA,geometry,hand_mean
681965,1646627,2976728.0,1300,7559.2166,204,GP,0,75.592166,2,"POLYGON ((432921.938 108920.374, 432931.875 10...",76.113777
681966,3588612,2976729.0,1300,6120.1665,204,GP,0,61.201665,2,"POLYGON ((435669.945 108166.760, 435723.973 10...",9.261046
681967,3588639,2976730.0,1300,5015.4655,204,GP,0,50.154655,2,"POLYGON ((435690.513 108261.462, 435639.992 10...",9.852235


### List comprehansion

In [6]:
shp = shape.copy()

In [7]:
%%time

# Open rasterio dataset
ds = rasterio.open("c:\\Users\\ncoz\\ARRS_susa\\fill_tif_fill\\dem_slo_03_HAND_fill_nan.tif")

# shp["hand_array"] = shp.geom.apply(lambda g: msk_rio(g, ds))
# shp["hand_mean"] = shp.hand_array.apply(lambda g: np.nanmean(g))

result = [np.nanmean(msk_rio(g, ds)) for g in shp["geom"]]
shp["hand_mean"] = result

ds.close()
shp = shp.drop(columns=["geom"])

C:\ProgramData\Anaconda3\envs\hand_index\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Wall time: 18min 9s


In [22]:
shp.tail(3)

,GERK_PID,POLJINA_ID,RABA_ID,POVR_GERK_,SIFRA_KMRS,OBRAZEC,INTERSECT,POVR_ar,REGIJA,geometry,hand_mean
4997,1425714,2119385.0,1100,5183.5005,006,GP,0,51.835005,5,"POLYGON ((566513.783 156266.730, 566511.996 15...",9.497029
4998,3319963,2119386.0,1300,5424.2931,204,GP,0,54.242931,5,"POLYGON ((566610.648 156430.799, 566627.841 15...",8.743620
4999,2352253,2119388.0,1100,7313.3327,206,GP,0,73.133327,5,"POLYGON ((566268.004 156617.728, 566294.294 15...",12.367579


## What if we need to extract from multiple images?


### Loop over all images with rasterio


Write a new function called `multi_img()` that will loop over all opened raster datasets and for each:
   - extract array from the raster
   - calculate mean and append it to a list
   - calculate aggregated mean and append at the end of the list

In [23]:
def multi_img(geom, raster):
    # Empty list for appending outputs
    out = []
    
    for item in raster:
        # Extract array
        out_image, _ = mask(item, geom, crop=True, all_touched=True, nodata=np.nan)
        # Calculate mean
        mean_arr = np.nanmean(out_image)
        # Append to list
        out.append(mean_arr)
        
    # Add aggregated mean at the end
    out.append(np.nanmean(out))
    
    return out

Reset table

In [24]:
shp = shape[:1000].copy()

Run the function over the entire table

In [27]:
%%time

img_paths = [
    "c:\\Users\\ncoz\\ARRS_susa\\ms_stacked\\20170220_SAVI_Cmask.tif",
    "c:\\Users\\ncoz\\ARRS_susa\\ms_stacked\\20170411_SAVI_Cmask.tif",
    "c:\\Users\\ncoz\\ARRS_susa\\ms_stacked\\20170421_SAVI_Cmask.tif"
]

# lambda
data_sources = [rasterio.open(a) for a in img_paths]
shp["temp_col"] = shp.geom.apply(lambda g: multi_img(g, data_sources))
[a.close() for a in data_sources]

# list to columns
column_names = ["slo_01", "slo_02", "slo_03", "slo_mean"]
shp[column_names] = gpd.GeoDataFrame(shp["temp_col"].tolist(), index=shp.index)
shp = shp.drop(columns=["temp_col"])

<ipython-input-23-5259dcb6260e>:9: RuntimeWarning: Mean of empty slice
  mean_arr = np.nanmean(out_image)


Wall time: 12 s


In [30]:
shp[column_names].head()

,slo_01,slo_02,slo_03,slo_mean
0,0.129386,NaN,0.347085,0.238235
1,0.098749,NaN,0.211576,0.155163
2,0.177642,NaN,0.399359,0.288501
3,0.195160,NaN,0.420483,0.307821
4,0.237651,0.436733,0.501296,0.391893


### Pre-stack all images into a single (multiband) file

Before we start processing, all images are stacked into a single file. The main bottleneck in this problem is reading data from files. Reading and writing from multiple files significantly increases the computation time. Possible work around is to create a file with multiple bands beforhand.

Creating such file takes couple of minutes and it can save hours in processing. However, the down side is that we need to have enough RAM to oprate with very large multi-band files.

**IMPORTANT!**
Be careful in what order you stack the individual images. If using pthname matching modules such as `glob`, the order of listed paths might be arbitrary and needs to be sorted (by filename/date) before stacking into single file.

In [32]:
%%time

file_list = [
    "c:\\Users\\ncoz\\ARRS_susa\\ms_stacked\\20170220_SAVI_Cmask.tif",
    "c:\\Users\\ncoz\\ARRS_susa\\ms_stacked\\20170411_SAVI_Cmask.tif",
    "c:\\Users\\ncoz\\ARRS_susa\\ms_stacked\\20170421_SAVI_Cmask.tif"
]
bnds = len(file_list)

# Read metadata of first file
with rasterio.open(file_list[0]) as src0:
    meta = src0.meta
    rst_shape = src0.shape

# Update meta to reflect the number of layers
meta.update(count=bnds)

# Create empty array
stack = np.empty((len(file_list), rst_shape[0], rst_shape[1]), meta["dtype"])

# Read directly into array
for b, fp in enumerate(file_list):
    with rasterio.open(fp) as ds:
        stack[b, :, :] = ds.read(1)

# Save file
stacked_file = "c:\\Users\\ncoz\\ARRS_susa\\ms_stacked\\stacked.tif"

with rasterio.open(stacked_file, 'w', **meta) as dst:
    dst.write(stack)

Wall time: 51.2 s


Write a new function called `stacked_img()` that will:
   - extract array from the raster
   - calculate mean for each band and store it as list
   - calculate aggregated mean and append at the end of the list

In [33]:
def stacked_img(geom, raster):
    # Extract raster
    out_image, _ = mask(raster, geom, crop=True, all_touched=True, nodata=np.nan)
    
    # Calculate mean for each band and store to list
    means = list(np.nanmean(out_image, axis=(1, 2)))
            
    # Add aggregated mean
    means.append(np.nanmean(means))
    
    return means

Reset table

In [34]:
shp = shape[:1000].copy()

Run the function over the entire table

In [35]:
%%time

# lambda
ds = rasterio.open(stacked_file)
shp["temp_col"] = shp.geom.apply(lambda g: stacked_img(g, ds))
ds.close()

# list to columns
column_names = ["slo_01", "slo_02", "slo_03", "slo_mean"]
shp[column_names] = gpd.GeoDataFrame(shp["temp_col"].tolist(), index=shp.index)
shp = shp.drop(columns=["temp_col"])

<ipython-input-33-bec6064d2e45>:6: RuntimeWarning: Mean of empty slice
  means = list(np.nanmean(out_image, axis=(1, 2)))


Wall time: 4.03 s


In [37]:
shp[column_names].head()

,slo_01,slo_02,slo_03,slo_mean
0,0.129386,NaN,0.347085,0.238235
1,0.098749,NaN,0.211576,0.155163
2,0.177642,NaN,0.399359,0.288501
3,0.195160,NaN,0.420483,0.307821
4,0.237651,0.436733,0.501296,0.391893
